In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [29]:
data = pd.read_csv('Data 2023 Januari Cleaned.csv')
data

,Unnamed: 0,Latitude,Longitude,Azimuth,Altitude,Dates,Time
0,0,-7.9271,112.6331,77.194752,-30.312057,2023-01-01,00:00:00
1,1,-7.9271,112.6331,78.406839,-29.841815,2023-01-01,00:05:00
2,2,-7.9271,112.6331,79.612169,-29.369533,2023-01-01,00:10:00
3,3,-7.9271,112.6331,80.810971,-28.895433,2023-01-01,00:15:00
4,4,-7.9271,112.6331,82.003476,-28.419729,2023-01-01,00:20:00
...,...,...,...,...,...,...,...
8636,8636,-7.9271,112.6331,67.294165,-28.127098,2023-01-30,23:40:00
8637,8637,-7.9271,112.6331,68.543122,-27.680523,2023-01-30,23:45:00
8638,8638,-7.9271,112.6331,69.785217,-27.230125,2023-01-30,23:50:00
8639,8639,-7.9271,112.6331,71.020630,-26.776134,2023-01-30,23:55:00


In [30]:
X = data[['Azimuth', 'Altitude']].values
y = data[['Azimuth', 'Altitude']].values
time = data['Time'].values

In [31]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [32]:
X_train, X_test, y_train, y_test, time_train, time_test = train_test_split(X, y, time, test_size=0.2, random_state=42)

In [33]:
def create_sequences(data, sequence_length):
    sequances=[]
    targets=[]
    time_sequances= []
    for i in range(len(data)-sequence_length):
        seq = data[i:i+sequence_length]
        target = data[i+sequence_length:i+sequence_length+1]
        time_seq = time[i:i+sequence_length]
        sequances.append(seq)
        targets.append(target)
        time_sequances.append(time_seq)
    return np.array(sequances), np.array(targets), np.array(time_sequances)

sequance_length = 10
X_train_sequences, y_train_sequences, time_train_sequences = create_sequences(X_train, sequance_length)
X_test_sequences, y_test_sequences, time_test_sequences = create_sequences(X_test, sequance_length)


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

class CoordinatePredictionRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(CoordinatePredictionRNN, self).__init()
        self.rnn = nn.LSTM(input_dim + 1, hidden_dim, num_layers, batch_first=True)
        self.fc_azimuth = nn.Linear(hidden_dim, 1)  # Output neuron for azimuth prediction
        self.fc_altitude = nn.Linear(hidden_dim, 1)  # Output neuron for altitude prediction

    def forward(self, x, time_seq):
        out, _ = self.rnn(x)
        azimuth = self.fc_azimuth(out[:, -1, :])
        altitude = self.fc_altitude(out[:, -1, :])
        return azimuth, altitude

input_dimension = X_train_sequences.shape[2]
hidden_dimension = 64  # You can adjust this
num_layers = 2  # You can adjust this
model = CoordinatePredictionRNN(input_dimension, hidden_dimension, num_layers)

criterion_azimuth = nn.MSELoss()
criterion_altitude = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


AttributeError: 'super' object has no attribute '_CoordinatePredictionRNN__init'